In [0]:
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from pyspark.sql.functions import lit
import math
import time
import tensorflow as tf

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-1452121680382276>, line 10
      8 import math
      9 import time
---> 10 import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [0]:
!pip install tensorflow
!pip install vaderSentiment

In [0]:
tweets_df = spark.table("combined_tweets")
price_df = spark.table("nasdaq_100_data_from_2010")
num_rows = price_df.count()
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 271680


In [0]:
from pyspark.sql.functions import when, col
# Replace "GOOG" with "GOOGL" in the ticker_symbol column
tweets_df = tweets_df.withColumn("ticker_symbol", 
                                       when(col("ticker_symbol") == "GOOG", "GOOGL")
                                       .otherwise(col("ticker_symbol")))

In [0]:
from pyspark.sql.functions import col

# Assuming the name of your DataFrame is tweet_df and the relevant column is named 'ticker_symbol'
googl_tweets_df = tweets_df.filter(col("ticker_symbol") == "GOOGL")
googl_price_df = price_df.filter(col("Name") == "GOOGL")

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import unicodedata

def sentiment_analysis(text):
    if text is not None:
        sentence = unicodedata.normalize('NFKD', text)
        sentiment_analyzer = SentimentIntensityAnalyzer()
        sentiment = sentiment_analyzer.polarity_scores(sentence)
        return sentiment['compound'], sentiment['neg'], sentiment['neu'], sentiment['pos']
    else:
        return None, None, None, None

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, StructType, StructField

# Define the schema of the return type
schema = StructType([
    StructField("sentiment_score", FloatType(), False),
    StructField("Negative", FloatType(), False),
    StructField("Neutral", FloatType(), False),
    StructField("Positive", FloatType(), False)
])

# Register the UDF
sentiment_udf = udf(sentiment_analysis, schema)


In [0]:
from pyspark.sql.functions import col

# Apply the UDF to the DataFrame
sentiment_df = googl_tweets_df.withColumn("sentiment", sentiment_udf(col("body")))

# Extract the sentiment fields into separate columns
sentiment_df = sentiment_df.select("*", 
                                   col("sentiment").getField("sentiment_score").alias("sentiment_score"),
                                   col("sentiment").getField("Negative").alias("Negative"),
                                   col("sentiment").getField("Neutral").alias("Neutral"),
                                   col("sentiment").getField("Positive").alias("Positive"))

# Drop the intermediate struct column
sentiment_df = sentiment_df.drop("sentiment")


In [0]:
sentiment_df.show()


+-------------------+----------+--------------------+-------------+-------------------+-----------+---------------+--------+-------+--------+
|           tweet_id| post_date|                body|ticker_symbol|    datetime_column|date_column|sentiment_score|Negative|Neutral|Positive|
+-------------------+----------+--------------------+-------------+-------------------+-----------+---------------+--------+-------+--------+
|1007582659105771520|1529061400|#Allergan 's Casf...|        GOOGL|2018-06-15 11:16:40| 2018-06-15|        -0.5837|   0.089|  0.911|     0.0|
|1007669321185812480|1529082062|Today's million d...|        GOOGL|2018-06-15 17:01:02| 2018-06-15|            0.0|     0.0|    1.0|     0.0|
|1007687510548287488|1529086399|CRN Exclusive: Go...|        GOOGL|2018-06-15 18:13:19| 2018-06-15|          0.128|     0.0|  0.889|   0.111|
|1007792550172692480|1529111442|#Coversin will ma...|        GOOGL|2018-06-16 01:10:42| 2018-06-16|         0.3182|     0.0|  0.956|   0.044|
|10080

In [0]:
sentiment_df2 = sentiment_df.drop('tweet_id','post_date','body','datetime_column','Negative','Neutral','Positive')
#sentiment_df2.show()

In [0]:
from pyspark.sql.functions import to_date, avg

# Group by ticker_symbol and date, then calculate the average sentiment_score
avg_sentiment_df = sentiment_df2.groupBy("date_column") \
                                .agg(avg("sentiment_score").alias("average_sentiment"))


In [0]:
googl_price_df.show()
num_rows = googl_price_df.count()
print("Number of rows in the DataFrame:", num_rows)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|      Date|              Open|              High|               Low|             Close|         Adj Close|  Volume| Name|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|2010-01-04| 313.7887878417969|  315.070068359375|312.43243408203125| 313.6886901855469| 313.6886901855469| 3908488|GOOGL|
|2010-01-05| 313.9039001464844| 314.2342224121094| 311.0810852050781|312.30731201171875|312.30731201171875| 6003391|GOOGL|
|2010-01-06| 313.2432556152344| 313.2432556152344| 303.4834899902344| 304.4344482421875| 304.4344482421875| 7949443|GOOGL|
|2010-01-07| 305.0050048828125| 305.3052978515625| 296.6216125488281|297.34735107421875|297.34735107421875|12815771|GOOGL|
|2010-01-08| 296.2962951660156|301.92694091796875|  294.849853515625| 301.3113098144531| 301.3113098144531| 9439151|GOOGL|
|2010-01-11|302.

In [0]:
from pyspark.sql.functions import col

# Join the DataFrames on both ticker_symbol and date
merged_df = googl_price_df.join(
    avg_sentiment_df,
    (googl_price_df.Date == avg_sentiment_df.date_column),
    "inner"
)

In [0]:
num_rows = merged_df.count()
print("Number of rows in the DataFrame:", num_rows)
merged_df.first()


Number of rows in the DataFrame: 1258


Row(Date=datetime.date(2018, 8, 10), Open=1259.1800537109375, High=1261.199951171875, Low=1247.1600341796875, Close=1252.510009765625, Adj Close=1252.510009765625, Volume=1330100, Name='GOOGL', date_column=datetime.date(2018, 8, 10), average_sentiment=0.14421530375489136)

In [0]:
merged_df.show()

+----------+------------------+------------------+------------------+------------------+------------------+-------+-----+-----------+--------------------+
|      Date|              Open|              High|               Low|             Close|         Adj Close| Volume| Name|date_column|   average_sentiment|
+----------+------------------+------------------+------------------+------------------+------------------+-------+-----+-----------+--------------------+
|2018-08-10|1259.1800537109375| 1261.199951171875|1247.1600341796875| 1252.510009765625| 1252.510009765625|1330100|GOOGL| 2018-08-10| 0.14421530375489136|
|2019-05-08|   1177.2900390625|  1184.27001953125|1167.6300048828125| 1170.780029296875| 1170.780029296875|1276000|GOOGL| 2019-05-08| 0.18862287500255967|
|2016-03-01| 721.2999877929688| 742.3300170898438| 718.6400146484375| 742.1699829101562| 742.1699829101562|3005600|GOOGL| 2016-03-01|0.009091039144992829|
|2019-06-04| 1044.489990234375|  1058.43994140625| 1036.030029296875| 

In [0]:
num_rows = merged_df.count()
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 1258


In [0]:
merged_df=merged_df.drop('date_column','Volume')

In [0]:
merged_df.show()

+----------+------------------+------------------+------------------+------------------+------------------+-----+--------------------+
|      Date|              Open|              High|               Low|             Close|         Adj Close| Name|   average_sentiment|
+----------+------------------+------------------+------------------+------------------+------------------+-----+--------------------+
|2018-08-10|1259.1800537109375| 1261.199951171875|1247.1600341796875| 1252.510009765625| 1252.510009765625|GOOGL| 0.14421530375489136|
|2019-05-08|   1177.2900390625|  1184.27001953125|1167.6300048828125| 1170.780029296875| 1170.780029296875|GOOGL| 0.18862287500255967|
|2016-03-01| 721.2999877929688| 742.3300170898438| 718.6400146484375| 742.1699829101562| 742.1699829101562|GOOGL|0.009091039144992829|
|2019-06-04| 1044.489990234375|  1058.43994140625| 1036.030029296875| 1054.489990234375| 1054.489990234375|GOOGL| 0.16858812500722706|
|2017-08-11| 923.7100219726562| 933.3599853515625|  921

In [0]:
merged_df=merged_df.drop('Name')

In [0]:
from pyspark.sql.functions import asc
sorted_df = merged_df.orderBy(asc("Date"))


In [0]:
sorted_df.show()

+----------+------------------+-----------------+------------------+------------------+------------------+-------------------+
|      Date|              Open|             High|               Low|             Close|         Adj Close|  average_sentiment|
+----------+------------------+-----------------+------------------+------------------+------------------+-------------------+
|2015-01-02| 532.5999755859375|535.7999877929688| 527.8800048828125| 529.5499877929688| 529.5499877929688| 0.1551390659296885|
|2015-01-05| 527.1500244140625| 527.989990234375|            517.75| 519.4600219726562| 519.4600219726562| 0.3075649142968205|
|2015-01-06|             520.5|521.2100219726562|505.54998779296875| 506.6400146484375| 506.6400146484375|0.30266869737769586|
|2015-01-07|510.95001220703125| 511.489990234375| 503.6499938964844| 505.1499938964844| 505.1499938964844|  0.261703954587489|
|2015-01-08|  501.510009765625|            507.5| 495.0199890136719| 506.9100036621094| 506.9100036621094|0.212

In [0]:
display(sorted_df)

In [0]:
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import IntegerType

# UDF to determine sentiment direction
sentiment_direction_udf = udf(lambda x: 1 if x > 0 else -1, IntegerType())

# Adding sentiment direction to DataFrame
sorted_df = sorted_df.withColumn('SentimentDirection', sentiment_direction_udf(col('average_sentiment')))

# Calculating stock movement and normalizing it
sorted_df = sorted_df.withColumn('StockMovement', (col('Close') - col('Open')) / col('Open'))
sorted_df = sorted_df.withColumn('NormalizedStockMovement', when(col('StockMovement') > 1, 1).otherwise(when(col('StockMovement') < -1, -1).otherwise(col('StockMovement'))))

# UDF to determine stock direction
stock_direction_udf = udf(lambda x: 1 if x > 0 else -1, IntegerType())

# Adding stock direction to DataFrame
sorted_df = sorted_df.withColumn('StockDirection', stock_direction_udf(col('NormalizedStockMovement')))

# Comparing sentiment and stock direction
sorted_df = sorted_df.withColumn('PredictionResult', when(col('SentimentDirection') == col('StockDirection'), 1).otherwise(0))

# Calculate accuracy
accuracy_df = sorted_df.agg({"PredictionResult": "mean"})
accuracy = accuracy_df.collect()[0][0]

print(f"Prediction Accuracy: {accuracy:.2f}")


Prediction Accuracy: 0.51
